# Choropleth overlay generator

This notebook creates high resolution images of choropleth maps using [GeoPandas](https://geopandas.org/en/stable/getting_started/install.html). These images can be added as a texture in Unreal Engine 5 and configured as an overlay material on static mesh actors to give the effect of a choropleth map in 3d.

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Load sample data - check geopandas docs for file formats: https://geopandas.org/en/stable/docs/user_guide/io.html
data_path = 'sample_data/FourLADs_2011LOAC.gpkg'
gdf = gpd.read_file(data_path)

# Quick CRS check
if gdf.crs != 'EPSG:27700':
    print('Check your coordinate reference system. This notebook was developed using EPSG:27700 and it may not work correctly with other coordinate reference systems.')

# Dictionary to add description to supergroup labels
legend_label_lookup = {'A': 'Intermediate Lifestyles', 
                       'B': 'High Density and High Rise Flats', 
                       'C': 'Settled Asians', 
                       'D': 'Urban Elites', 
                       'E': 'City Vibe', 
                       'F': 'London Life-Cycle', 
                       'G': 'Multi-Ethnic Suburbs', 
                       'H': 'Ageing City Fringe'}

column_to_plot = 'Super'
legend_filename = 'images/FourLADs_LOAC_legend.png'
texture_filename = 'images/FourLADs_LOAC_texture.png'

ue_world_origin_bng_eastings = 532816
ue_world_origin_bng_northings = 180759


In [ ]:
def export_legend(legend, filename, legend_label_dict=None, expand=[-5,-5,5,5]):
    '''
    Takes a matplotlib legend, draws the full figure to which that legend belongs, 
    cuts out the legend and saves the legend as a separate file. 
    This legend can then be added as a heads-up display UI element in your unreal project.
    See https://docs.unrealengine.com/5.2/en-US/umg-ui-designer-quick-start-guide/
    '''
    fig  = legend.figure
    fig.canvas.draw()

    if legend_label_dict is not None:
        for label in legend.get_texts():
            label.set_text(legend_label_dict[label.get_text()])
    
    bbox  = legend.get_window_extent()
    bbox = bbox.from_extents(*(bbox.extents + np.array(expand)))
    bbox = bbox.transformed(fig.dpi_scale_trans.inverted())

    fig.savefig(filename, dpi=600, bbox_inches=bbox)

In [ ]:

# Set image size. Here we've doubled the default figure size.
# It doesn't matter too much what size is used, as long as the resulting image is high-res enough 
# (which can also be configured via the DPI setting) 
width_inches = 12.8
height_inches = 9.6
dpi = 600

fig = plt.figure(figsize=(width_inches, height_inches))
ax = fig.add_subplot(111)

# Plot figure
gdf.plot(column_to_plot, 
         ax=ax, 
         categorical=True, 
         legend=True,
         legend_kwds={'bbox_to_anchor': (2.0,1)}, # Add enough space for the to fall outside the main graph
         linewidth=0)

# Extract legend and save as a separate file
legend = ax.get_legend()
export_legend(legend, filename=legend_filename, legend_label_dict=legend_label_lookup)

# Since we've exported the legend, we can now remove it.
legend.remove()

# Now we need to amend the axis and padding to ensure we can map the pixels to locations on the map
ax.axis('scaled')
xlim = ax.get_xlim()
ylim = ax.get_ylim()
ax.set_xticks([])
ax.set_yticks([])
ax.axis('off')
plt.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0, wspace=0)

# If you suspect the padding might be causing issues, then it can be helpful to plot a point in the lower
# left corner for debugging purposes - a quarter circle should extend to the limit of the image with no white space
# ax.plot(ax.get_xlim()[0], ax.get_ylim()[0], 'ro', ms=100)

plt.savefig(texture_filename, bbox_inches='tight', pad_inches=0., dpi=dpi)

In [ ]:
print('The following information is needed to build the overlay texture in unreal:')
print('LL eastings offset: %s' % ax.get_xlim()[0])
print('LL northings offset: %s' % ax.get_ylim()[0])

print('Eastings world origin to texture start (cm): %s' % (-100 * (ue_world_origin_bng_eastings - ax.get_xlim()[0])))
print('Northings world origin to texture start (cm): %s' % (100 * (ue_world_origin_bng_northings - ax.get_ylim()[0])))
print('Texture width (cm): %s' % (100 * (ax.get_xlim()[1] - ax.get_xlim()[0])))
print('Text height (cm): %s' % (100 * (ax.get_ylim()[1] - ax.get_ylim()[0])))


Import then texture into Unreal Engine, then a new material can be created with a blueprint as shown in the image below. 

This material stretches the image of the map over the Unreal worldspace for the relevant area shown in the texture map.

![image](images/Overlay_material_example.PNG)

If everything's worked, then a plane in Unreal Engine with the material assigned should show a tiling of the map where the tile overlaying the origin point has map coordinates that match up with unreal world coordinates in line with your chosen anchor point:

![image](images/Choropleth_overlay_plane.PNG)

Adding other static mesh actors to unreal, such as city buildings, and then assigning the material to these buildings as an overlay material can provide an interesting option for 3d data visualisation:

![image](images/choropleth_overlay_example.PNG)